### Import dependecies

In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import time
import traceback
import re
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

### Initialize File Paths

In [2]:
input_file_path = '/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/output/data_draft.csv'
output_file_path = '/Users/sauravyadav/Documents/Repos/Datasets/Drinking_Surveys/output/data_draft_v2.csv'

In [3]:
df = pd.read_csv(input_file_path)

### TO DO
- Create Missed Survey Records
    - For every Survey, there should be 12 follow up surveys

### 1. Create Missed Survey Records

In [4]:
df.head(2)

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvNamePlus,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,UI_DrnkFin,UI_DrnkNum,DrnkNum,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur
0,2001,1,1,Thu,5ab44c822c9b2f5baaa6aa80,Submission,5a8381835893ca7070656612,After Your 1st Drink,Start,Assessment,2018-03-22,17:52:06,2018-03-22,20:38:26,2018-03-22,05:00:00,1.0,1.0,NaN,59.0,34.0,22.0,35.0,37.0,33.0,54.0,42.0,40.0,28.0,1.0,CONDITION_SKIPPED,1.0,3,3,CONDITION_SKIPPED,62,64,34,74,30,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,9.0,6.0
1,2001,1,1,Thu,d0274600-d886-4a59-9c8e-8bcde51902de,Missed,5a8381835893ca7070656612,Drinking Follow-Ups,Short,NaN,2018-03-22,21:08:26,2018-03-22,21:18:26,2018-03-22,21:08:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Helper functions to create a new row

In [5]:
def get_next_date_time_day(prev_datetime_str: str):
    new_datetime = datetime.strptime(prev_datetime_str, '%Y-%m-%d %H:%M:%S') + timedelta(minutes=30)

    new_date_str = new_datetime.strftime('%Y-%m-%d')
    new_time_str = new_datetime.strftime('%H:%M:%S')
    new_day = new_datetime.strftime('%a')
    
    return new_date_str, new_time_str, new_day
    
def get_next_row(prev_row, isFirst = False):

    if isFirst:
        prev_datetime_str = prev_row['SubDate'] + ' ' + prev_row['SubTime']
    else:
        prev_datetime_str = prev_row['TrigDate'] + ' ' + prev_row['TrigTime']
    
    new_date_str, new_time_str, new_day = get_next_date_time_day(prev_datetime_str)
    
    new_row = {'SubID' : prev_row['SubID'], 'Burst' : prev_row['Burst'],
                  'Weekend' : prev_row['Weekend'], 'Day' : new_day,
                'RespType' : 'Missed (Synthetic)', 'UserID' : prev_row['UserID'], 'SurvName' : 'Drinking Follow-Ups',
                'TrigDate': new_date_str, 'TrigTime' : new_time_str}
    
    if prev_row['SurvNamePlus'] == 'Start' or prev_row['SurvNamePlus'] == 'Long':
        new_row['SurvNamePlus'] = 'Short'
    else:
        new_row['SurvNamePlus'] = 'Long'
        
    return new_row
    

### Iterative technique

In [6]:

# expected_trigger_time = None
# prev_row = None
# i = 0
# row_list = []
# session_row_count = 0

# while i < len(df):
#     row = df.iloc[i]

#     if row['SurvName'] != 'After Your 1st Drink' and session_row_count >= 13:
#         print(f'check: {i}')
        
#     # First of the session OR Last Session Over
#     if row['SurvName'] == 'After Your 1st Drink' or session_row_count == 13:
        
#         # Finish previous session
#         while session_row_count > 0 and session_row_count < 13:
#             new_row = get_next_row(prev_row)
#             row_list.append(new_row)
#             prev_row = new_row
#             session_row_count += 1


#         # if row['SurvName'] == 'After Your 1st Drink' and row['RespType'] == 'Missed':
#         #     i += 1
#         #     continue
            
#         expected_trigger_time = datetime.strptime(row['SubTime'], '%H:%M:%S') + timedelta(minutes=30)
#         i += 1
#         session_row_count = 1
#         prev_row = row
#         row_list.append(row.to_dict())
#         continue
        

#     # if record is present
#     elif datetime.strptime(row['TrigTime'], '%H:%M:%S') - expected_trigger_time > timedelta(minutes=5):
#         new_row = get_next_row(prev_row)
#         row_list.append(new_row)
#         prev_row = new_row
#         session_row_count += 1
#         if session_row_count > 12 :
#             print(f'{i} and {len(row_list)-1}')

#     else:
#         prev_row = row
#         row_list.append(row)
#         session_row_count += 1
#         i += 1
    
#     expected_trigger_time = expected_trigger_time + timedelta(minutes=30)


## Overlap Analysis

In [7]:
session_start_df = df[df['SurvName'] == 'After Your 1st Drink'].reset_index()
overlap_count = 0
affected_participants = set()
response_ids = set()
evidence_rows = []

for index, row in tqdm(session_start_df.iloc[:-1].iterrows(), total=len(session_start_df) - 1):
    subsequent_row = session_start_df.iloc[index + 1]
    
    prev_dt_string = row['SubDate'] + ' ' + row['SubTime']
    prev_dt = datetime.strptime(prev_dt_string, '%Y-%m-%d %H:%M:%S')

    next_dt_string = subsequent_row['SubDate'] + ' ' + subsequent_row['SubTime']
    next_dt = datetime.strptime(next_dt_string, '%Y-%m-%d %H:%M:%S')
    
    if row['SubID'] == subsequent_row['SubID'] and next_dt - prev_dt < timedelta(hours = 6):
        overlap_count += 1
        affected_participants.add(row['SubID'])
        response_ids.add(row['RespID'])
        evidence_rows.append(row['index'])

print(f'Overlaps: {overlap_count} |||| affected_participants_count: {len(affected_participants)}')
print([x + 2 for x in evidence_rows])
print(f'Total sessions: {len(session_start_df)}')
print(f'Total participants: {df['SubID'].nunique()}')

100%|████████████████████████████████████| 4436/4436 [00:00<00:00, 20174.46it/s]

Overlaps: 42 |||| affected_participants_count: 24
[339, 365, 391, 393, 806, 812, 845, 848, 1333, 1422, 1999, 2003, 2004, 2008, 2027, 2029, 2048, 2049, 3039, 3618, 4562, 4646, 4979, 5416, 6148, 6257, 6283, 6286, 6439, 6504, 6861, 7050, 7167, 8510, 8511, 8562, 13435, 18124, 25618, 36408, 44722, 45686]
Total sessions: 4437
Total participants: 393


In [8]:
df.head()

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvNamePlus,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,UI_DrnkFin,UI_DrnkNum,DrnkNum,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur
0,2001,1,1,Thu,5ab44c822c9b2f5baaa6aa80,Submission,5a8381835893ca7070656612,After Your 1st Drink,Start,Assessment,2018-03-22,17:52:06,2018-03-22,20:38:26,2018-03-22,05:00:00,1.0,1.0,NaN,59.0,34.0,22.0,35.0,37.0,33.0,54.0,42.0,40.0,28.0,1.0,CONDITION_SKIPPED,1.0,3,3,CONDITION_SKIPPED,62,64,34,74,30,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,9.0,6.0
1,2001,1,1,Thu,d0274600-d886-4a59-9c8e-8bcde51902de,Missed,5a8381835893ca7070656612,Drinking Follow-Ups,Short,NaN,2018-03-22,21:08:26,2018-03-22,21:18:26,2018-03-22,21:08:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,1,1,Thu,5ab45b7d2c9b2f5baaa6bdc7,Submission,5a8381835893ca7070656612,Drinking Follow-Ups,Long,Assessment,2018-03-22,21:40:59,2018-03-22,21:42:20,2018-03-22,21:38:26,NaN,NaN,2.0,63,30,27.0,43.0,NaN,67.0,30.0,30.0,26.0,74.0,1.0,CONDITION_SKIPPED,1.0,4,"1,3",CONDITION_SKIPPED,77,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,71.0,0.0,0.0,70.0
3,2001,1,1,Thu,5ab462b076ea035ba07f51fd,Submission,5a8381835893ca7070656612,Drinking Follow-Ups,Short,Assessment,2018-03-22,22:12:13,2018-03-22,22:13:03,2018-03-22,22:08:26,NaN,NaN,1.0,38,16,16.0,40.0,NaN,34.0,54.0,22.0,27.0,70.0,1.0,CONDITION_SKIPPED,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001,1,1,Thu,c0db9c9b-18d2-4b37-a392-ebf22999964e,Missed,5a8381835893ca7070656612,Drinking Follow-Ups,Long,NaN,2018-03-22,22:38:26,2018-03-22,22:48:26,2018-03-22,22:38:26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Based on Number Matching 
- Our aim is to ensure that each session has 12 follow up surveys
- We create synthetic rows to meet this goal

In [9]:
row_list = []
info = []
session_start_df = df[df['SurvName'] == 'After Your 1st Drink']

for index, row in tqdm(session_start_df.iterrows(), total=len(session_start_df)):
    row_list.append(row.to_dict())
    
    if row['RespType'] == 'Missed':    
        # print(f'Check for missed first: {index}')
        continue

    follow_up_survey_count = 0
    dummy_row_count = 0
    # We are creating a synthetic row
    next_row = get_next_row(row, True) #follow_up_survey_count == 0)
    
    while follow_up_survey_count < 12:
        
        next_row_in_df = df[(df['SubID'] == row['SubID']) & (df['TrigDate'] == next_row['TrigDate']) & (df['TrigTime'] == next_row['TrigTime'])]
        if  len(next_row_in_df) == 1:
            next_row = next_row_in_df.iloc[0].to_dict()
            row_list.append(next_row)    
        elif len(next_row_in_df) == 0:
            dummy_row_count += 1
            row_list.append(next_row)
        else:
            # print(f'Check for duplicate submission time: {next_row_in_df.iloc[0].name}')
            submitted_survey_df = next_row_in_df[next_row_in_df['RespType'] == 'Submission']
            if len(submitted_survey_df)>0:
                next_row = submitted_survey_df.iloc[0].to_dict()
            else:
                next_row = next_row_in_df.iloc[0].to_dict()
            row_list.append(next_row)

        next_row =  get_next_row(next_row, False)
        follow_up_survey_count += 1

    if dummy_row_count > 0:
        info.append({'SubID': row['SubID'], 'Rows Added': dummy_row_count, 'TrigTime': row['TrigTime']})

100%|███████████████████████████████████████| 4437/4437 [02:20<00:00, 31.67it/s]


In [10]:
res_df = pd.DataFrame(row_list).reset_index(drop=True)
info_df = pd.DataFrame(info).reset_index(drop=True)

In [11]:
# res_df[res_df['SurvNamePlus'].isna()].head()
# res_df['SurvNamePlus'].value_counts()

In [12]:
def df_num(df):
    
    session_count = len(df[ (df['SurvName'] == 'After Your 1st Drink') & (df['RespType'] != 'Missed')])
    missed_first_count = len(df[(df['SurvName'] == 'After Your 1st Drink') & (df['RespType'] == 'Missed')] )
    
    print(f'Session Count: {session_count}')
    follow_up_survey_count = len(df[df['SurvName'] == 'Drinking Follow-Ups'])
    print(f'Follow_up_survey_count: {follow_up_survey_count}')
    
    print(f'Required rows: {session_count*12 - follow_up_survey_count}')

In [13]:
df_num(df)
df_num(res_df)

Session Count: 4009
Follow_up_survey_count: 39720
Required rows: 8388
Session Count: 4009
Follow_up_survey_count: 48108
Required rows: 0


### New Column: Drinking Bin

In [14]:
res_df['Drinking_bin'] = res_df.apply(lambda x: 1 if ((pd.notnull(x['DrnkNum']) and x['DrnkNum'] > 1)  or (x['SurvName'] == 'After Your 1st Drink' and x['RespType'] != 'Missed' and x['RespType'] != 'Missed (Synthetic)')) else (0 if (pd.notnull(x['DrnkNum']) and x['DrnkNum'] == 1 and x['RespType'] != 'Missed'and x['RespType'] != 'Missed (Synthetic)') else ''), axis = 1)

### New Columns: 'DrnkNum_combine', 'DrnkNum_cumulative'

In [15]:
res_df['DrnkNum_combine'] = res_df.apply(lambda x: x['DrnkNum'] - 1 if (pd.notnull(x['DrnkNum']) and x['DrnkNum'] > 1 and x['RespType'] != 'Missed' and x['SurvName'] == 'Drinking Follow-Ups')  else (x['UI_DrnkNum'] if (x['SurvName'] == 'After Your 1st Drink' and x['RespType'] != 'Missed')  else (0 if (pd.notnull(x['DrnkNum']) and x['DrnkNum'] == 1 and x['RespType'] != 'Missed' and x['RespType'] != 'Missed (Synthetic)') else None)), axis = 1)
res_df['DrnkNum_combine'] = res_df['DrnkNum_combine'].astype('Int64')

In [16]:
grouped = res_df.groupby(['SubID', 'Weekend', 'Burst'])
res_df['DrnkNum_cumulative'] = None

for group_name, group_df in grouped:
    cumulativeDrinkNum = 0
    for index, row in group_df.iterrows():
        
        if row['SurvName'] == 'After Your 1st Drink':
            cumulativeDrinkNum = 0

        if row['RespType'] == 'Missed'  or row['RespType'] == 'Missed (Synthetic)':
            continue
            
        cumulativeDrinkNum += 0 if pd.isna(row['DrnkNum_combine']) else row['DrnkNum_combine']
        res_df.at[index, 'DrnkNum_cumulative'] = cumulativeDrinkNum

In [17]:
res_df = res_df[['SubID','Burst', 'Weekend', 'Day', 'RespID', 'RespType', 'UserID', 'SurvName', 'SurvNamePlus', 'SurvType',
       'InitDate', 'InitTime', 'SubDate', 'SubTime', 'TrigDate', 'TrigTime', 'Drinking_bin',
       'UI_DrnkFin', 'UI_DrnkNum', 'DrnkNum', 'DrnkNum_combine', 'DrnkNum_cumulative', 'Plsur', 'Rliev', 'IntxNow',
       'MorAlc', 'enrgz', 'excit', 'sedat', 'slotht', 'slug', 'up', 'crntloc',
       'othrloc', 'wthothr', 'NumOth', 'OthTyp', 'OthWho', 'EnjInt', 'ActCld',
       'ActAcc', 'OthCld', 'OthAcc', 'Vrtl', 'AlnLke', 'AlnGd', 'WshOth',
       'lnly', 'rjct', 'incl', 'acpt','vrtint', 'vrtoth',
        'vrtwho', 'vrtenj', 'vrtcld', 'VrtAcc', 'VrtOthCld',
       'VrtOthAcc', 'NotInt', 'WshInt', 'UI_NtInt_lnly', 'UI_NtInt_rjct',
       'UI_NtInt_incl', 'UI_NtInt_acpt', 'said', 'spent', 'impt', 'spur']]

In [18]:
res_df['SurvNamePlus'] = res_df['SurvNamePlus'].fillna('Dummy')

In [19]:
res_df.to_csv(output_file_path, index=False)

In [20]:
res_df.head(2)

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvNamePlus,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,Drinking_bin,UI_DrnkFin,UI_DrnkNum,DrnkNum,DrnkNum_combine,DrnkNum_cumulative,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur
0,2001,1,1,Thu,5ab44c822c9b2f5baaa6aa80,Submission,5a8381835893ca7070656612,After Your 1st Drink,Start,Assessment,2018-03-22,17:52:06,2018-03-22,20:38:26,2018-03-22,05:00:00,1,1.0,1.0,NaN,1,1,59.0,34.0,22.0,35.0,37.0,33.0,54.0,42.0,40.0,28.0,1.0,CONDITION_SKIPPED,1.0,3,3,CONDITION_SKIPPED,62,64,34,74,30,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,9.0,6.0
1,2001,1,1,Thu,d0274600-d886-4a59-9c8e-8bcde51902de,Missed,5a8381835893ca7070656612,Drinking Follow-Ups,Short,NaN,2018-03-22,21:08:26,2018-03-22,21:18:26,2018-03-22,21:08:26,,NaN,NaN,NaN,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
res_df[res_df['RespID'].isin(response_ids)].index + 2

Index([  327,   353,   379,   392,   769,   782,   808,   821,  1276,  1367,
        1939,  1952,  1965,  1978,  1991,  2004,  2017,  2030,  2927,  3486,
        4370,  4435,  4734,  5150,  5852,  5956,  5982,  5995,  6138,  6203,
        6515,  6697,  6801,  8036,  8049,  8088, 12742, 17461, 25014, 39281,
       50101, 51083],
      dtype='int64')

In [22]:
len(res_df[res_df['RespID'].isin(response_ids)])

42

In [23]:
len(evidence_rows)

42

In [26]:
res_df.head()

,SubID,Burst,Weekend,Day,RespID,RespType,UserID,SurvName,SurvNamePlus,SurvType,InitDate,InitTime,SubDate,SubTime,TrigDate,TrigTime,Drinking_bin,UI_DrnkFin,UI_DrnkNum,DrnkNum,DrnkNum_combine,DrnkNum_cumulative,Plsur,Rliev,IntxNow,MorAlc,enrgz,excit,sedat,slotht,slug,up,crntloc,othrloc,wthothr,NumOth,OthTyp,OthWho,EnjInt,ActCld,ActAcc,OthCld,OthAcc,Vrtl,AlnLke,AlnGd,WshOth,lnly,rjct,incl,acpt,vrtint,vrtoth,vrtwho,vrtenj,vrtcld,VrtAcc,VrtOthCld,VrtOthAcc,NotInt,WshInt,UI_NtInt_lnly,UI_NtInt_rjct,UI_NtInt_incl,UI_NtInt_acpt,said,spent,impt,spur
0,2001,1,1,Thu,5ab44c822c9b2f5baaa6aa80,Submission,5a8381835893ca7070656612,After Your 1st Drink,Start,Assessment,2018-03-22,17:52:06,2018-03-22,20:38:26,2018-03-22,05:00:00,1,1.0,1.0,NaN,1,1,59.0,34.0,22.0,35.0,37.0,33.0,54.0,42.0,40.0,28.0,1.0,CONDITION_SKIPPED,1.0,3,3,CONDITION_SKIPPED,62,64,34,74,30,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,0.0,0.0,9.0,6.0
1,2001,1,1,Thu,d0274600-d886-4a59-9c8e-8bcde51902de,Missed,5a8381835893ca7070656612,Drinking Follow-Ups,Short,NaN,2018-03-22,21:08:26,2018-03-22,21:18:26,2018-03-22,21:08:26,,NaN,NaN,NaN,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2001,1,1,Thu,5ab45b7d2c9b2f5baaa6bdc7,Submission,5a8381835893ca7070656612,Drinking Follow-Ups,Long,Assessment,2018-03-22,21:40:59,2018-03-22,21:42:20,2018-03-22,21:38:26,1,NaN,NaN,2.0,1,2,63,30,27.0,43.0,NaN,67.0,30.0,30.0,26.0,74.0,1.0,CONDITION_SKIPPED,1.0,4,"1,3",CONDITION_SKIPPED,77,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,71.0,0.0,0.0,70.0
3,2001,1,1,Thu,5ab462b076ea035ba07f51fd,Submission,5a8381835893ca7070656612,Drinking Follow-Ups,Short,Assessment,2018-03-22,22:12:13,2018-03-22,22:13:03,2018-03-22,22:08:26,0,NaN,NaN,1.0,0,2,38,16,16.0,40.0,NaN,34.0,54.0,22.0,27.0,70.0,1.0,CONDITION_SKIPPED,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CONDITION_SKIPPED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2001,1,1,Thu,c0db9c9b-18d2-4b37-a392-ebf22999964e,Missed,5a8381835893ca7070656612,Drinking Follow-Ups,Long,NaN,2018-03-22,22:38:26,2018-03-22,22:48:26,2018-03-22,22:38:26,,NaN,NaN,NaN,<NA>,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# NEW:

# [769,   782,   808,   821,  1276,  1367,
#         1939,  1952,  1965,  1978,  1991,  2004,  2017,  2030,  2927,  3486,
#         4370,  4435,  4734,  5150,  5852,  5956,  5982,  5995,  6138,  6203,
#         6515,  6697,  6801,  8036,  8049,  8088, 12742, 17461, 25014, 39281,
#        50101, 51083]




# OLD:
# [769,   782,   808,   821,  1276,  1367, 1939,  1952,  1965,  1978,  1991,  2004,  2017,  2030,  2927,  3486, 4370,  4435,  4734,  5150,  5852,  5956,  5982,  5995,  6138,  6203,
#         6769,  6951,  7055,  8290,  8303,  8342, 13126, 17845, 25652, 40800, 53222, 54815]